In [1]:
%env DATA_DIR=$HOME/datasets
%env EXPERIMENT_BASE=$HOME/experiments/ood_flows
%env LOG_LEVEL=INFO
%env BATCH_SIZE=64
%env OPTIM_LR=0.001
%env OPTIM_M=0.8
%env TRAIN_EPOCHS=100
%env EXC_RESUME=1
%env DATASET_NAME=QPM_species
%env MANIFOLD_D=512
%env MODEL_NAME=resnet

env: DATA_DIR=$HOME/datasets
env: EXPERIMENT_BASE=$HOME/experiments/ood_flows
env: LOG_LEVEL=INFO
env: BATCH_SIZE=64
env: OPTIM_LR=0.001
env: OPTIM_M=0.8
env: TRAIN_EPOCHS=100
env: EXC_RESUME=1
env: DATASET_NAME=QPM_species
env: MANIFOLD_D=512
env: MODEL_NAME=resnet


In [2]:
import lightning.pytorch as pl
import numpy as np
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers.wandb import WandbLogger

from config import Config, load_config
from datasets import get_data
from models import get_model

In [3]:
# initialize the RNG deterministically
np.random.seed(42)
torch.manual_seed(42)
torch.set_float32_matmul_precision('medium')

config = load_config()

# initialize data attributes and loaders
get_data(config)
config.print_labels()

assert config.datamodule
model = get_model(config)

In [4]:
assert config.datamodule
config.datamodule.setup("train")
config.datamodule.setup("val")
config.datamodule.setup("test")

Dataset type train label type: species
Loaded 375443 images
Dataset type val label type: species
Loaded 48562 images
Dataset type test label type: species
Loaded 48790 images
Performing ind/ood split


100%|██████████| 375443/375443 [00:00<00:00, 1753593.92it/s]


Train - OK


100%|██████████| 48562/48562 [00:00<00:00, 1645982.83it/s]


Val - OK


100%|██████████| 48790/48790 [00:00<00:00, 1629183.36it/s]

Test - OK
Performed ind/ood split
375443 48562 48790 0


In [5]:
train_loader = config.datamodule.train_dataloader()
val_loader = config.datamodule.val_dataloader()
test_loader = config.datamodule.test_dataloader()

In [16]:
from models.common import edl_probs
from tqdm import tqdm

# accuracy.reset()
# classifier_loss = "edl"
for batch_idx, batch in enumerate(tqdm(test_loader)):
    x, y = batch
    print(x.size(), y.size())
    break

  0%|          | 0/763 [00:00<?, ?it/s]

torch.Size([64, 60, 60, 1]) torch.Size([64])
